In [1]:
import os
from collections import defaultdict, Counter
import csv
import re
import time
import pandas as pd
import ipywidgets as widget

In [2]:
option = widget.Dropdown(options=['Economic', 'Environmental', 'Social', 'Sustainability', 'All'], value= 'All', description='Option:')
display(option)

Dropdown(description='Option:', index=4, options=('Economic', 'Environmental', 'Social', 'Sustainability', 'Al…

In [3]:
typeOfDictionaries = ['Economic', 'Environmental', 'Social', 'Sustainability']

In [4]:
def reg_tokenize(regFilter, text):
    text = text.lower()
    text = re.sub(r"[^A-Za-z—\-\'\’ ]", ' ', text)
    text = re.sub(r"\s+", ' ', text)
    words = re.findall(regFilter, text)
    return words

In [5]:
def findTxts(path):
    result = list()
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith('.txt'):
                filePath = open('TextFiles/'+file,
                                'r', encoding='utf-8')
                text = filePath.read()
                result.append(text)
    return result

In [6]:
def createPattern(type):
    if type == 'Sustainability':
        df = pd.DataFrame(columns=['words'])
        for dictionaryName in [n for n in typeOfDictionaries if n != 'Sustainability']:
            dfExtra = pd.read_excel("SustainabilityDictionaries\\" + dictionaryName + "dictionary.xlsx", sheet_name='Ark1', names=['words'])
            df = pd.concat([df, dfExtra], ignore_index=True)
        wordList = df['words'].to_list()
        return "|".join((wordList))
    df = pd.read_excel(
        "SustainabilityDictionaries\\" + type + "dictionary.xlsx", sheet_name='Ark1', names=['words'])
    wordList = df['words'].to_list()
    return "\\b"+ "\\b|\\b".join((wordList)) + "\\b"

Sustainability stands for all dictionaries in one.

In [7]:
%%time
if option.value == 'All':
    for dictionaryName in typeOfDictionaries:
        WORD = re.compile(createPattern(dictionaryName))
        token_list = list()
        for txtFile in findTxts('TextFiles'):
            tokens = reg_tokenize(WORD, txtFile)
            token_list.extend(tokens)
        count = Counter(token_list)
        with open("Counts/" + dictionaryName + "WordFreqDict.csv", "w+", newline="", encoding='utf-8') as csv_file:
            csv_file.write("%s,%s\n" % ('word', 'count'))
            for key, value in count.most_common():
                csv_file.write("%s, %s\n" % (key, value))
else:
    WORD = re.compile(createPattern(option.value))
    token_list = list()
    for txtFile in findTxts('TextFiles'):
        tokens = reg_tokenize(WORD, txtFile)
        token_list.extend(tokens)
    count = Counter(token_list)
    with open("Counts/" + option.value + "WordFreqDict.csv", "w+", newline="", encoding='utf-8') as csv_file:
        csv_file.write("%s,%s\n" % ('word', 'count'))
        for key, value in count.most_common():
            csv_file.write("%s, %s\n" % (key, value))

Wall time: 1min 49s
